<table style="width: 100%; margin-left: auto; margin-right: auto;" border="0">
    <tr>
        <td rowspan="3"> <img src="brasao_ufrn.png" width="150"/> </td>
        <td style="text-align: center">  Escola de Ciências e Tecnologia </td>
        <td rowspan="3" style="text-align: center"> UFRN<br> CT<br> PPGEMECA </td>
    </tr>
    <tr>
        <td style="text-align: center"> PPGEMECA - Percepção Robótica </td>
    </tr>
    <tr>
        <td>
            <p style="text-align: left;">Prof. Bruno Silva<br>
                                         Prof. Marcelo Nogueira</p>
        </td>
    </tr>
</table>

## Reconstrução Estéreo I

> Reconstrução estéreo: obter as coordenadas 3D de uma cena a partir de um par de imagens estéreo

Permite que sejam obtidas a distância e direção de *features* presentes na imagem.

## Reconstrução Estéreo II

Assumindo que as câmeras foram calibradas e possuem as imagens retificadas,
a reconstrução se dá por:

1. Obtenção de correspondências: um mesmo ponto é achado na imagem esquerda e direita
2. Cálculo da disparidade (diferença na coordenada $x$ na imagem de um mesmo ponto)
3. Triangulação: cálculo da profundidade $Z_c$ a partir da disparidade e parâmetros extrínsecos
4. Cálculo de $X_c$ e $Y_c$ dado $Z_c$

## Reconstrução Estéreo III

Observe à esquerda um exemplo de par estéreo retificado: após retificação, os planos das duas imagens são paralelos e as linhas epipolares de ambas as imagens coincidem, para facilitar o processo de
busca por correspondências. A retificação corta e "entorta" a imagem, além de introduzir regiões nulas.

À direita, um exemplo de reconstrução 3D a partir de imagens estéreo.

<table border="0">
<tr>
<td><img src="28_reconstrucao_estereo/stereo_rectification.png" style="margin:auto; width: 600px;"/></td>
<td><img src="28_reconstrucao_estereo/stereo_reconstruction.png" style="margin:auto; width: 600px;"/></td>
</tr>
<tr>
<td colspan="2">Imagens da documentação da OpenCV </td>
</tr>
</table>

## Reconstrução Estéreo IV

Este documento mostra como obter uma reconstrução estéreo
a partir de imagens estéreo retificadas e calibradas.

São exemplos deste tipo de imagens as imagens de odometria
visual do *dataset* KITTI [4]

## Correspondência Estéreo I

O primeiro passo para obter-se uma reconstrução estéreo é computar correspondências
entre pontos da imagem esquerda e da imagem direita.

Isto é realizado por meio de algoritmos do tipo *block matching* (casamento por blocos):
dado uma janela (bloco) da imagem esquerda, a sua correspondência é buscada na imagem
direita "deslizando-se" o bloco na imagem direita, até que a similaridade seja a maior.

## Correspondência Estéreo II

**Mapa de disparidade**: imagem que contém, em cada uma de suas posições, o valor da disparidade (em pixels).

Um mapa de disparidade é computado pelos algoritmos de correspondência estéreo.

## Correspondência Estéreo III

Na biblioteca OpenCV, estão presentes pelo menos 2 algoritmos para correspondência
estéreo:

1. `StereoBM`: *block matching* simples $\rightarrow$ baixo custo computacional mas com mapa resultante de precisão limitada
2. `StereoSGBM`: *semi global block matching* $\rightarrow$ resulta em mapas de qualidade superior a um maior custo computacional

## Correspondência Estéreo - Funções da OpenCV I

`StereoBM.create`: cria um objeto do tipo *block matching* para computar correspondências

Parâmetros:

- `numDisparities`: número de disparidades esperado. Precisa ser múltiplo de 16
- `blockSize`: tamanho da janela de busca. Precisa ser ímpar e maior do que 5

`compute`: faz o objeto computar correspondências entre duas imagens (esquerda e direita, nesta ordem) fornecidas

In [ ]:
'''
Calcula correspondência estéreo
para um par de imagens retificadas
utilizando Block Matching simples.
'''

import numpy as np
import cv2
from matplotlib import pyplot as plt

def main():

    img_l = cv2.imread('kitti00_00l.png', cv2.IMREAD_GRAYSCALE)
    img_r = cv2.imread('kitti00_00r.png', cv2.IMREAD_GRAYSCALE)

    num_disparities = 96
    block_size = 11
    matcher = cv2.StereoBM.create(numDisparities=num_disparities, blockSize=block_size)
    
    img_disp = matcher.compute(img_l, img_r)
    plt.imshow(img_disp, 'gray')
    plt.show()

if __name__ == '__main__':
    main()

## Correspondência Estéreo - Funções da OpenCV II

`StereoSGBM.create`: cria um objeto do tipo *semi global block matching* para computar correspondências

Parâmetros:

- `numDisparities`: número de disparidades esperado. Precisa ser múltiplo de 16
- `minDisparity`: disparidade mínima esperada
- `blockSize`: tamanho da janela de busca. Precisa ser ímpar
- `P1`: penalidade da disparidade por mudança de mais ou menos 1 entre pixels vizinhos. Quanto maior o valor, mais suave será a disparidade
- `P2`: penalidade da disparidade por mudança maior que 1 entre pixels vizinhos. Quanto maior o valor, mais suave será a disparidade
- `mode`: utilizar `cv2.STEREO_SGBM_MODE_SGBM_3WAY`

`compute`: faz o objeto computar correspondências entre duas imagens (esquerda e direita, nesta ordem) fornecidas

In [ ]:
'''
Calcula correspondência estéreo
para um par de imagens retificadas
utilizando Semi Global Block Matching.
'''

import numpy as np
import cv2
from matplotlib import pyplot as plt

def main():

    img_l = cv2.imread('kitti00_00l.png', cv2.IMREAD_GRAYSCALE)
    img_r = cv2.imread('kitti00_00r.png', cv2.IMREAD_GRAYSCALE)

    sad_window = 6
    num_disparities = sad_window*16
    block_size = 11
    matcher = cv2.StereoSGBM.create(numDisparities=num_disparities,
                                    minDisparity=0,
                                    blockSize=block_size,
                                    P1 = 8 * 3 * sad_window ** 2,
                                    P2 = 32 * 3 * sad_window ** 2,
                                    mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
                                    )
    
    img_disp = matcher.compute(img_l, img_r)
    plt.imshow(img_disp, 'gray')
    plt.show()

if __name__ == '__main__':
    main()

## Disparidade

A imagem abaixo exibe o mapa de disparidade resultante para as imagens de exemplo
com o algoritmo SGBM.

<table border="0">
<tr>
<td><img src="28_reconstrucao_estereo/disparity.png" style="margin:auto; width: 800px;"/></td>
</tr>
</table>

## Correspondência Estéreo - Funções da OpenCV III

Em comum a ambas as funções:

- Parametrizar os algoritmos é uma tarefa necessária para cada aplicação específica
- Parametrizar os algoritmos é feito em geral por tentativa e erro (ou uso de interface gráfica)

## Triangulação I

A triangulação é o processo que resulta na profundidade de um ponto
dada a sua disparidade:

$$
Z = \frac{fT}{d},
$$

onde:

- $f$ é a distância focal
- $F$ é a linha de base (distância entre as câmeras)
- $d$ é a disparidade

**Como as imagens estão retificadas, a disparidade é calculada como sendo
a diferença da coordenada x esquerda menos a da direita**
$\rightarrow d = x^r-x^l$

## Triangulação II

**Mapa de profundidade**: imagem que contém, em cada uma de suas posições, o valor da profundidade (em unidades de distância).

## Triangulação - Código

O código a seguir mostra como obter um mapa de profundidade a partir
da imagem de disparidade.

A OpenCV possui uma função para isto (`reprojectTo3D`) mas não iremos utilizá-la.

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

def compute_disparity(img_l, img_r):
    sad_window = 6
    num_disparities = sad_window*16
    block_size = 11
    matcher = cv2.StereoSGBM.create(numDisparities=num_disparities,
                                    minDisparity=0,
                                    blockSize=block_size,
                                    P1 = 8 * 3 * sad_window ** 2,
                                    P2 = 32 * 3 * sad_window ** 2,
                                    mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
                                    )
    
    return matcher.compute(img_l, img_r).astype(np.float32)/16    

def compute_depth(f, T, img_disp):
    img_disp[img_disp == 0.0] = 0.1 # evita divisões por 0
    img_disp[img_disp == -1.0] = 0.1 # evita valores inválidos na disparidade

    img_depth = np.zeros(img_disp.shape)
    return (f * T)/img_disp

def main():

    T = 0.54
    f = 718.856

    img_l = cv2.imread('kitti00_00l.png', cv2.IMREAD_GRAYSCALE)
    img_r = cv2.imread('kitti00_00r.png', cv2.IMREAD_GRAYSCALE)
    
    img_disp = compute_disparity(img_l, img_r)
    img_depth = compute_depth(f, T, img_disp)
    print(f'min depth: {img_depth.min()}')
    print(f'max depth: {img_depth.max()}')

if __name__ == '__main__':
    main()

## Nuvem de Pontos I

**Nuvem de pontos**: conjunto de pontos (2D ou 3D), é a
forma mais comum do resultado de uma reconstrução 3D

## Nuvem de Pontos II

Para obter as coordenadas 3D de um determinado ponto (ref. ao Sistema de Coordenadas de câmera),
é preciso conhecer a sua profundidade $Z$. A partir dela, realiza-se o processo inverso da projeção:

$$
x = \frac{f_x}{Z_c} X_c + c_x
\Rightarrow
X_c = \frac{Z_c(x - c_x)}{f_x}
$$
e
$$
y = \frac{f_y}{Z_c} Y_c + c_y
\Rightarrow
Y_c = \frac{Z_c(y - c_y)}{f_y}
$$
onde $x$ e $y$ são as coordenadas de um ponto na imagem com profundidade $Z_c$

## Mapa de Profundidade I

A imagem abaixo exibe a nuvem de pontos resultante para as imagens
de exemplo. Observe que ela contém muitos pontos com a profundidade
máxima calculada (em vermelho).

<table border="0">
<tr>
<td><img src="28_reconstrucao_estereo/cloud_all.png" style="margin:auto; width: 800px;"/></td>
</tr>
</table>

## Mapa de Profundidade II

A imagem abaixo exibe a nuvem de pontos após filtragem:
os pontos com profundidade máxima foram modificados
para conter $Z_c = 0$.

<table border="0">
<tr>
<td><img src="28_reconstrucao_estereo/cloud_filtered.png" style="margin:auto; width: 800px;"/></td>
</tr>
</table>

## Reconstrução 3D: Código/prática

O código a seguir está estruturado para exibir uma nuvem de pontos
3D computada a partir de um par de imagens estéreo.

Implemente as funcionalidades que estão faltando como parte do projeto final.

import numpy as np
import cv2
from matplotlib import pyplot as plt

def compute_disparity(img_l, img_r):
    sad_window = 6
    num_disparities = sad_window*16
    block_size = 11
    matcher = cv2.StereoSGBM.create(numDisparities=num_disparities,
                                    minDisparity=0,
                                    blockSize=block_size,
                                    P1 = 8 * 3 * sad_window ** 2,
                                    P2 = 32 * 3 * sad_window ** 2,
                                    mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
                                    )
    
    return matcher.compute(img_l, img_r).astype(np.float32)/16    

def compute_depth(f, T, img_disp):
    img_disp[img_disp == 0.0] = 0.1 # evita divisões por 0
    img_disp[img_disp == -1.0] = 0.1 # evita valores inválidos na disparidade

    img_depth = np.zeros(img_disp.shape)
    return (f * T)/img_disp

def reconstruct_point(K, x, y, Z):
    fx = K[0,0]
    fy = K[1,1]
    cx = K[0,2]
    cy = K[1,2]

    #utilize a inversa da projeção para computar cada ponto 3D
    #X = ???
    #Y = ???
    return np.array([X, Y, Z]).T

def generate_point_cloud(K, img_depth):
    max_depth = img_depth.max()
    cloud = []
    for i in range(img_depth.shape[0]):
        for j in range(img_depth.shape[1]):
            Z = img_depth[i, j]
            if Z == max_depth: # pontos com profundidade máxima recebem Z = 0 (possivelmente inválidos)
                Z = 0
            pt3D = reconstruct_point(K, j, i, Z)
            cloud.append(pt3D)

    return np.array(cloud)

def view_point_cloud(points3D):
    # implemente uma maneira de visualizar uma nuvem de pontos
    # (matplotlib ou open3D)

def main():

    T = 0.54
    f = 718.856
    K = np.array([[718.856,       0, 607.192],
                  [      0, 718.856, 185.215],
                  [      0,       0,       1]])

    img_l = cv2.imread('kitti00_00l.png', cv2.IMREAD_GRAYSCALE) #aloeL.jpg
    img_r = cv2.imread('kitti00_00r.png', cv2.IMREAD_GRAYSCALE)
    
    img_disp = compute_disparity(img_l, img_r)
    img_depth = compute_depth(f, T, img_disp)
    point_cloud = generate_point_cloud(K, img_depth)
    view_point_cloud(point_cloud)

if __name__ == '__main__':
    main()

## Sumário

Nesta aula - reconstrução estéreo:

- Correspondência estéreo
- Triangulação com imagens retificadas
- Reconstrução de imagens em nuvens de pontos 3D

## Referências:

[1] R. Szeliski. 2010. Computer Vision: Algorithms and Applications (1st. ed.). Springer.

[2] E. Trucco and A. Verri. 1998. Introductory Techniques for 3-D Computer Vision. Prentice Hall PTR, USA.

[3] A. Kaehler and G. Bradski. 2014. Learning OpenCV, 2nd Edition. O'Reilly Media, Inc.

[4] KITTI Vision Benchmark Suite.<br>
    Disponível em https://www.cvlibs.net/datasets/kitti/eval_odometry.php